In [1]:
import pandas as pd
import os
from datetime import datetime

ROOT_DIR =  os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

In [2]:
sources = ["AP","CBS","CNN","DailyMail","Express","Fox","Guardian", "Mirror", "NYT", "Reuters"]
windows = [5]
dfs = []

for source in sources:
    df = pd.read_csv(os.path.join(ROOT_DIR,"results_5",f"{source}_results.csv"),index_col="Date")
    df = df.sort_values(by="Resonance",ascending=False)
    dfs.append(df)

data = dict(zip(sources,dfs))

In [5]:
data["Reuters"].head(5)

,URL,Title,Text,Novelty,Transience,Resonance,Topic
Date,,,,,,,
2022-04-03,https://www.reuters.com/article/us-ukraine-cri...,Satellite images show long trench at Ukrainian...,Satellite images show a 45-foot-longtrench dug...,7.184148,4.813597,2.370552,18
2022-04-03,https://www.reuters.com/article/us-ukraine-cri...,U.N. chief calls for independent probe of civi...,United Nations Secretary-General Antonio Guter...,6.446977,4.507633,1.939344,18
2022-04-03,https://www.reuters.com/article/us-ukraine-cri...,"Ukraine says 410 bodies found near Kyiv, witne...",Ukraine has found 410 bodies in towns near Kyi...,6.311095,4.397720,1.913375,18
2022-04-03,https://www.reuters.com/article/us-ukraine-cri...,Russia asks U.N. Security Council to discuss B...,Russia has requested that the United Nations S...,7.120142,5.333942,1.786200,28
2022-10-07,https://www.reuters.com/article/ukraine-crisis...,Drone crashes into Russian military airfield n...,A drone has crashed into a military airfield i...,6.729309,4.959994,1.769315,27
